In [1]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
!pip install chromadb llama-index-vector-stores-chroma


  Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached llama_index_vector_stores_chroma-0.4.2-py3-none-any.whl.metadata (413 bytes)
  Using cached pybase64-1.4.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.22.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.6 kB)
  Using cached opentelemetry_api-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.35.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 M

In [2]:
import os
import warnings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from huggingface_hub import notebook_login


In [3]:
try:
    notebook_login()
except Exception as e:
    print(f"Hugging Face login failed: {e}")


In [4]:
def download_file(url, filename):
    """Download file with error handling"""
    try:
        !wget {url} -O {filename}
        print(f"Successfully downloaded {filename}")
        return True
    except Exception as e:
        print(f"Failed to download {filename}: {e}")
        return False


In [5]:
data_files = []
if download_file("https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/majmaolbayan.txt", "/content/majmaolbayan.txt"):
    data_files.append("/content/majmaolbayan.txt")

if download_file("https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/alborhan.txt", "/content/alborhan.txt"):
    data_files.append("/content/alborhan.txt")

if not data_files:
    raise Exception("No data files were successfully downloaded")


--2025-07-19 09:50:44--  https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/majmaolbayan.txt
Resolving filedn.eu (filedn.eu)... 185.62.236.186
Connecting to filedn.eu (filedn.eu)|185.62.236.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18917253 (18M) [text/plain]
Saving to: ‘/content/majmaolbayan.txt’

/content/majmaolbay 100%[===================>]  18.04M  4.85MB/s    in 3.7s    

2025-07-19 09:50:49 (4.85 MB/s) - ‘/content/majmaolbayan.txt’ saved [18917253/18917253]

Successfully downloaded /content/majmaolbayan.txt
--2025-07-19 09:50:50--  https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/alborhan.txt
Resolving filedn.eu (filedn.eu)... 185.62.236.186
Connecting to filedn.eu (filedn.eu)|185.62.236.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23839945 (23M) [text/plain]
Saving to: ‘/content/alborhan.txt’

/content/alborhan.t 100%[===================>]  22.74M  7.36MB/s    in 3.1s    

2025-07-19 09:50:54

In [6]:
CHUNK_SIZE = 512
CHUNK_OVERLAP = 100
PERSIST_DIR = "/content/my_chroma_index"
COLLECTION_NAME = "my_collection"
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"


In [7]:
print("Loading documents...")
try:
    documents = SimpleDirectoryReader(input_files=data_files).load_data()
    print(f"Loaded {len(documents)} documents")
except Exception as e:
    print(f"Error loading documents: {e}")
    raise


Loading documents...
Loaded 2 documents


In [8]:
print("Initializing embedding model...")
try:
    embed_model = HuggingFaceEmbedding(model_name=EMBED_MODEL_NAME)
    print("Embedding model loaded successfully")
except Exception as e:
    print(f"Error loading embedding model: {e}")
    raise


Initializing embedding model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded successfully


In [9]:
node_parser = SentenceSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)


In [10]:
print("Setting up ChromaDB...")
try:
    chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
    chroma_collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    print("ChromaDB setup complete")
except Exception as e:
    print(f"Error setting up ChromaDB: {e}")
    raise


Setting up ChromaDB...
ChromaDB setup complete


In [11]:
print("Creating vector index...")
try:
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        embed_model=embed_model,
        node_parser=node_parser,  # Now actually using the node parser
        show_progress=True
    )
    print("Index created successfully")
except Exception as e:
    print(f"Error creating index: {e}")
    raise


Creating vector index...


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/248 [00:00<?, ?it/s]

Index created successfully


In [12]:
print("Creating zip file...")
try:
    !zip -r /content/my_chroma_index.zip /content/my_chroma_index
    print("Zip file created")
except Exception as e:
    print(f"Error creating zip file: {e}")

Creating zip file...
  adding: content/my_chroma_index/ (stored 0%)
  adding: content/my_chroma_index/chroma.sqlite3 (deflated 65%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/ (stored 0%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/index_metadata.pickle (deflated 44%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/data_level0.bin (deflated 12%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/header.bin (deflated 57%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/length.bin (deflated 84%)
  adding: content/my_chroma_index/3e701ba8-5c03-4cf1-98ab-2e7246a46626/link_lists.bin (deflated 81%)
Zip file created


In [13]:
try:
    from google.colab import files
    files.download('/content/my_chroma_index.zip')
    print("Index downloaded successfully")
except Exception as e:
    print(f"Error downloading file: {e}")

print("Index preparation complete!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Index downloaded successfully
Index preparation complete!
